In [2]:
import numpy as np
import matplotlib.pyplot as plt

# 1. Load and combine data

**Data information**
- The first entry corresponds to the head tip. 
- The last 3 correspond to the tail tip. 
- The timestep is $\Delta t = 1/5s $

In [22]:
# 1. Load raw data and combine: 
for name in ["video2", "video3"]:
    combined_data = []
    for xy in ["x", "y"]:
        filename = f"raw/{name}_{xy}.csv"
        sub_data = np.loadtxt(filename , delimiter = ",")
        print(f"{filename=}: {sub_data.shape=}")
        combined_data.append(sub_data)
    combined_data = np.array(combined_data).T
    
    # save: 
    combined_filename = f"combined/{name}.npy"
    np.save(combined_filename, combined_data)
    print(f"{combined_filename=}: {combined_data.shape=}\n")

filename='raw/video2_x.csv': sub_data.shape=(24, 1938)
filename='raw/video2_y.csv': sub_data.shape=(24, 1938)
combined_filename='combined/video2.npy': combined_data.shape=(1938, 24, 2)

filename='raw/video3_x.csv': sub_data.shape=(24, 2999)
filename='raw/video3_y.csv': sub_data.shape=(24, 2999)
combined_filename='combined/video3.npy': combined_data.shape=(2999, 24, 2)


# 2. Preprocess data (centering): 

- Take the combined data and use only one tail point (instead of 3). 
- at each time step, subtract the center of mass of each skelleton point. 

In [32]:
for name in ["video2", "video3"]:
    # load combined data: 
    combined_data = np.load(f"combined/{name}.npy")
    
    # remove two of the three tail points: 
    combined_data_one_tail = combined_data[:, :-2, :]
    print(f"{combined_data_one_tail.shape=}")
    
    # remove center of mass for each frame: 
    center_of_mass = np.mean(combined_data_one_tail, axis=1)
    print(f"{center_of_mass.shape=}")
    
    nr_of_body_points = combined_data_one_tail.shape[1]  # should be 22.
    centered_data = combined_data_one_tail - np.repeat(center_of_mass[:, np.newaxis, :], repeats=nr_of_body_points, axis=1)
    
    print(f"{centered_data.shape=}")
    preprocessed_filename = f"centered/{name}_centered.npy"
    np.save(preprocessed_filename, centered_data)
    

combined_data_one_tail.shape=(1938, 22, 2)
center_of_mass.shape=(1938, 2)
centered_data.shape=(1938, 22, 2)
combined_data_one_tail.shape=(2999, 22, 2)
center_of_mass.shape=(2999, 2)
centered_data.shape=(2999, 22, 2)


# 3. Make head and Tail always in one line. 

- head and tail will always be on a horizontal line with the head on the right side (positive x axis) and tail on the left (negativ x axis). 

In [24]:
for name in ["video2", "video3"]:
    print(f"{name=}")
    # load combined data: 
    centered_data = np.load(f"centered/{name}_centered.npy")
    print(f"{centered_data.shape=}")
    
    head = centered_data[:, 0, :]
    tail = centered_data[:, -1, :]
    diff = head - tail
    dx = diff[:, 0]
    dy = diff[:, 1]
    
    # rotation matrix: 
    angles = np.arctan(dy/dx)
    # print(angles)
    
    rotation_matrix = np.array([[np.cos(angles), -np.sin(angles)],
                           [np.sin(angles), np.cos(angles)]])
    print(f"{rotation_matrix.shape=}")
    
    rotated_points = np.zeros(centered_data.shape)
    
    for i_frame,  points in enumerate(centered_data):
        for i_point, point in enumerate(points):
            rotated_point = np.dot(point, rotation_matrix[:, :, i_frame])
            rotated_points[i_frame, i_point, :] = rotated_point
    
    rotated_filename = f"rotated/{name}_rotated.npy"
    np.save(rotated_filename, rotated_points)
    

name='video2'
centered_data.shape=(1938, 22, 2)
rotation_matrix.shape=(2, 2, 1938)
name='video3'
centered_data.shape=(2999, 22, 2)
rotation_matrix.shape=(2, 2, 2999)
